In [1]:
#!pip install learn2learn
#!pip install torchmeta
import torch
import utils
import model as MODEL
import train as TRAIN
import matplotlib.pyplot as plt
from google.colab import drive
import epoch
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__ == '__main__':
    Quincy = False ## set true to run Quincy's model
    if Quincy ==True: ## param search in this file doe snot support this 
        Params = {'datasize':96,'nways':32,'kshots':1,'in_channels':3,
                'num_classes':45,'hidden_size':2, "innerStep":.0006,
                'MetaLR':.0006,"number_of_tasks":1, "Order":True,"outerVSinner": 1,
                "epoch":50,'aug':True, 'trainvalSplit':.5,'trainvalSplit': .8, 'Quincy':True}
    else:
        Params = {'Training loss': 126.29095458984375, 'Training Error Rate': 0.3249491751194,
                  'Validation topk': 0.7814275622367859, 'Validation Error Rate': 0.5703125, 
                  'datasize': 96, 'nways': 5, 'kshots': 1, 'in_channels': 3, 'num_classes': 45,
                  'hidden_size': 2, 'innerStep': 0.0006, 'MetaLR': 0.0006, 'number_of_tasks': 4, 
                  'Order': True, 'outerVSinner': 1, 'epoch': 50, 'aug': True, 'Img_size': 96, 
                  'bs': 32, 'c_dim': 45, 'trainsz': 17718, 'trainvalSplit':.5,'trainvalSplit': .8, 'Quincy':False}
    # print('Getting data')
    train_tasks,val_loader,test_loader,Params=utils.colabgetdata(Params)

   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/utils.py:229: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xtest = torch.tensor(test_data).permute(0, 3, 1, 2)


In [ ]:
Params, trainacc, trainloss, valacc, valtopk = epoch.epochthrough(train_tasks,Params,val_loader)


epoch: 0 val accuracy:  0.0865885391831398  topk:  0.23046875
epoch: 1 val accuracy:  0.1477864533662796  topk:  0.326171875
epoch: 2 val accuracy:  0.16015625  topk:  0.314453125
epoch: 3 val accuracy:  0.1673177033662796  topk:  0.3522135317325592
epoch: 4 val accuracy:  0.203125  topk:  0.392578125
epoch: 5 val accuracy:  0.193359375  topk:  0.388671875
epoch: 6 val accuracy:  0.2096354216337204  topk:  0.4029947817325592
epoch: 7 val accuracy:  0.2467447966337204  topk:  0.4524739682674408
epoch: 8 val accuracy:  0.234375  topk:  0.4342447817325592
epoch: 9 val accuracy:  0.2578125  topk:  0.431640625
epoch: 10 val accuracy:  0.255859375  topk:  0.4361979067325592
epoch: 11 val accuracy:  0.265625  topk:  0.48046875
epoch: 12 val accuracy:  0.2623697817325592  topk:  0.4739583432674408
epoch: 13 val accuracy:  0.2506510317325592  topk:  0.4739583432674408


In [ ]:
loss_rate = valacc[-1]
reduce = [True,False]
nwayss =[32,20,10,5]
for i in reduce:
    Params['Order']=i
    for tasknum in range(4): ## number of tasks inside each outer loop
        Params["number_of_tasks"] = int(2**tasknum)
        for InoutRatio in range(2): ## ratio of data in outer loop of task to one of the data in tasks right now it is 1:1, 2:1, 3:1
            Params['outerVSinner'] = InoutRatio+1
            for meta in range(4):## learning rates
                Params['MetaLR'] *=(1**(-meta+1))
                for innLR in range(4):
                    Params["innerStep"] *=(1**(-innLR+1))
                    for nwayys in nwayss:
                        Params['nways']=nwayys
                    model = MODEL.ConvolutionalNeuralNetwork(Params)
                    model = model.to(device)
                    Atrain = 0
                    Ltrain  =0
                    Aval = 0
                    Akval = 0
                    for i in range(Params['epoch']):
                        Atrain, Ltrain = TRAIN.train(model, train_tasks,Params, val_loader)
                        Aval, Akval = utils.getvalErr(model, val_loader)
                        if ((i <= 5) and Aval < .1): ## not to waste time stop early
                             break
                        if Aval > loss_rate: ## if our val is better then what we had save it
                            print("found one better")
                            print(loss_rate)
                            print(Params)
                            dict = {'Training loss': Ltrain, "Training Error Rate": Atrain, "Validation topk": Akval,
                                    "Validation Error Rate": Aval}
                            tofile = {**dict, **Params}
                            with open('/content/drive/MyDrive/Project/BestSoFar.txt', 'a') as f:
                                print(tofile, file=f)
                                loss_rate = dict["Validation Error Rate"]